In [3]:
using LinearAlgebra
using Plots
import Base: getproperty, \, show

In [4]:


struct LU_Fac{T<:Real}
    lu::Matrix{T}
    p::Array{Int, 1}
end

function getproperty(F::LU_Fac, d::Symbol)
    
    if d === :L
        return UnitLowerTriangular(F.lu)
    elseif d === :U
        return UpperTriangular(F.lu)
    else
        getfield(F, d)
    end
end

function propertynames(F::LU_Fac, private::Bool=false)
    properties = (:L, :U)
    if private
        return (fieldnames(typeof(F))..., properties...)
    else
        return properties
    end
end

function show(io::IO, mime::MIME{Symbol("text/plain")}, F::LU_Fac)
    print(io, "L = ")
    show(io, mime, F.L)
    print(io, "\n\nU = ")
    show(io, mime, F.U)
end


show (generic function with 346 methods)

In [10]:

function lu_factorization!(A::AbstractMatrix{T}) where T <: Real
    n = min(size(A)...)
    piv = collect(1:n)
    
    @inbounds begin
        for k = 1:n-1
            # find pivot element
            pivot = k
            max_elem = abs(A[pivot, k])
            for j = k+1:n
                row_elem = abs(A[j, k])
                if row_elem > max_elem
                    max_elem = row_elem
                    pivot = j
                end
            end
            if k != pivot
                piv[k], piv[pivot] = piv[pivot], piv[k]

                # change rows
                for j = 1:n
                    A[pivot, j], A[k, j] = A[k, j], A[pivot, j]
                end
            end
                 
            Akkinv = inv(A[k,k])
            for j = k+1:n
                A[j,k] *= Akkinv
            end
            for l = k+1:n
                for j = k+1:n
                    A[j,l] -= A[j,k] * A[k,l]
                end
            end
        end
    end
    return LU_Fac{T}(
        A,
        piv
    )
end


lu_factorization(A) = lu_factorization!(copy(A))


lu_factorization (generic function with 1 method)

In [6]:

@inline function forward_substitution(F::LU_Fac, b::AbstractArray)
    x = similar(b)
    bp = b[F.p]
    L = F.lu
    
    @inbounds begin
        x[1] = b[1]
        
        for k = 2:length(b)
            x[k] = b[k]

            for j = 1:k-1
                x[k] -= L[k,j] * x[j]
            end

        end
    end
    
    return x
end

@inline function backward_substitution(F::LU_Fac, b::AbstractArray)
    x = similar(b)
    n = length(b)
    U = F.lu
    
    @inbounds begin
        Uinv = inv(U[end,end])
        x[end] = b[end] * Uinv
        for k = (n - 1):-1:1
            x[k] = b[k] 

            for j = k+1:n
                x[k] -= U[k,j] * x[j]
            end
            Uinv = inv(U[k, k])
            x[k] *= Uinv
        end
        return x
    end
end


@inline function lu_solve(A::LU_Fac, b::AbstractArray{<:Real, 1})
    y = forward_substitution(A, b)
    x = backward_substitution(A, y)
end

function lu_solve(A::AbstractMatrix, b::AbstractArray)
    lu = lu_factorization(A)
    lu_solve(lu, b)
end

function lu_solve(A::LU_Fac, B::AbstractMatrix{<:Real})
    mapslices(B; dims=1) do b
        lu_solve(A, b)
    end    
end
    

\(A::LU_Fac, b::AbstractArray) = lu_solve(A, b);

In [8]:
n = 1000
A = rand(n,n)
b = A*ones(n, n)
@time F = lu_factorization(A)
@time x = F \ b

  0.719991 seconds (8 allocations: 7.637 MiB, 0.74% gc time)
  2.371217 seconds (10.51 k allocations: 31.069 MiB, 1.34% gc time)


1000×1000 Array{Float64,2}:
   60.8547     60.8547     60.8547   …    60.8547     60.8547     60.8547 
  -28.0113    -28.0113    -28.0113       -28.0113    -28.0113    -28.0113 
  -70.5221    -70.5221    -70.5221       -70.5221    -70.5221    -70.5221 
  -23.7275    -23.7275    -23.7275       -23.7275    -23.7275    -23.7275 
   33.8021     33.8021     33.8021        33.8021     33.8021     33.8021 
   29.5284     29.5284     29.5284   …    29.5284     29.5284     29.5284 
   84.4069     84.4069     84.4069        84.4069     84.4069     84.4069 
  -60.1389    -60.1389    -60.1389       -60.1389    -60.1389    -60.1389 
   19.2839     19.2839     19.2839        19.2839     19.2839     19.2839 
 -120.293    -120.293    -120.293       -120.293    -120.293    -120.293  
  -45.5952    -45.5952    -45.5952   …   -45.5952    -45.5952    -45.5952 
  -56.7375    -56.7375    -56.7375       -56.7375    -56.7375    -56.7375 
  -43.5873    -43.5873    -43.5873       -43.5873    -43.5873    -43.587

In [11]:
A = [1 2 3 4 
     2 3 4 2
     1 1 1 2
     0 2 1/3 3]
F = lu_factorization(copy(A))
@assert F.L *F.U ≈ A
display(F)
display(lu(A))
# b = [1, 1, 1, 1//1]
# x1 = forward_substitution(F, b)
# x2 = backward_substitution(F, b)

AssertionError: AssertionError: F.L * F.U ≈ A